## imports / setup

In [ ]:
import epics
import time
import numpy as np

## PVs

In [ ]:
pv_list_all = ['QUAD:LI26:201:BCTRL',
    'QUAD:LI26:301:BCTRL',
    'QUAD:LI26:401:BCTRL',
    'QUAD:LI26:501:BCTRL',
    'QUAD:LI26:601:BCTRL',
    'QUAD:LI26:701:BCTRL',
    'QUAD:LI26:801:BCTRL',
    'QUAD:LI26:901:BCTRL',

    'TDLY:LI21:1:ADelaySet', #1115.6*10**-9 +/- 7e-9
    'MKB:SYS0:3:VAL',  #-0.1 to 0.1
    'TDLY:LI21:1:CDelaySet', #1106.1*10**-9 +/- 7e-9
    'MKB:SYS0:4:VAL', #-0.1 to 0.1
    'SIOC:SYS0:ML07:AO736', #pulse 1
    'SIOC:SYS0:ML07:AO737', #pulse 2
    #'SIOC:SYS0:ML07:AO738', #pulse 3
    #'SIOC:SYS0:ML07:AO739', #pulse 4

    'QUAD:LTUH:620:BCTRL',
    'QUAD:LTUH:640:BCTRL',
    'QUAD:LTUH:660:BCTRL',
    'QUAD:LTUH:680:BCTRL',

    'QUAD:LI21:221:BCTRL',
    'QUAD:LI21:251:BCTRL',
    'QUAD:LI24:740:BCTRL',
    'QUAD:LI24:860:BCTRL',
    'QUAD:LTUH:440:BCTRL',
    'QUAD:LTUH:460:BCTRL']

pv_list_read = ['DIAG:FEE1:202:241:Data',
                'SIOC:SYS0:ML07:AO736', #pulse 1
                'SIOC:SYS0:ML07:AO737',
                'GDET:FEE1:241:ENRC',
               'GUN:IN20:1:GUN_WF',
                'ACCL:IN20:300:L0A_WF',
                'ACCL:IN20:400:L0B_WF',
                'ACCL:LI21:1:L1S_WF',
                'ACCL:LI21:180:L1X_WF',
                'TCAV:DMPH:360:TCA_WF',
                'ACCL:LI24:100:KLY_WF',
                'ACCL:LI24:200:KLY_WF',
                'ACCL:LI24:300:KLY_EF' ]

In [ ]:
current_values = dict(zip(pv_list_read, epics.caget_many(pv_list_read)))
print(current_values)

### evaluate definition

In [ ]:
def do_measurement(inputs):

    sett = True
    # set values
    if sett is True:
        for name, val in inputs.items():

            if name == 'TDLY:LI21:1:ADelaySet':
                val=val*10**-5
                print(val)
            if name == 'TDLY:LI21:1:CDelaySet':
                val=val*10**-5
                print(val)
          # if name == 'MKB:SYS0:3:VAL':
          #     val=val*10**-6
          #     print(val)
          # if name == 'MKB:SYS0:4:VAL':
          #     val=val*10**-6
          #     print(val)
            epics.caput(name, val)

        ## wait
        time.sleep(8.0)


    # get measured values and settings
    data_list = epics.caget_many(pv_list_read)

    data = dict(zip(pv_list_read, data_list))

    # can replace this with new way of getting gas detector, but also don't technically need it if the individual pulse intensity metrics are working
    thresh = 0.0001
    obj = epics.caget('GDET:FEE1:241:ENRCHSTCUHBR')
    obj = obj[obj > thresh]
    counter = 0
    meanobj = np.nanmean(obj)
    while obj.shape[0] == 0 or meanobj <thresh:
        obj = epics.caget('GDET:FEE1:241:ENRCHSTCUHBR')
        obj = obj[obj > thresh]
        meanobj = np.nanmean(obj)
        counter = counter + 1
        #print('catching gdet < 0.1')
        time.sleep(0.0001)
        if counter > 200:
            break
    obj0 = np.nanmean(obj)

    #these are the 1st and 2nd pulse -- PV will change and need to get from Alex or have Alex change
    obj2 =  epics.caget('SIOC:SYS0:ML07:AO737')
    obj1 =  epics.caget('SIOC:SYS0:ML07:AO736')
    obj_sum = obj1 + obj2

    print('obj ', obj0)
    print('obj_sum ', obj_sum)
    print('obj1 ', obj1)
    print('obj2 ', obj2)


    #for including losses in constraints, calculate total losses, or look at charge
    #soft_cblm_indexes = range(26, 48)
    hard_cblm_indexes = range(13, 46)
    #soft_loss_PVS = [f"CBLM:UNDS:{ele}10:I1_LOSS" for ele in soft_cblm_indexes]
    hard_loss_PVS = [f"CBLM:UNDH:{ele}75:I1_LOSS" for ele in hard_cblm_indexes]

    data["TMITH"] = epics.caget("BPMS:LI30:201:TMITCUH1H") / 1e9
    #data["TMITS"] = epics.caget("BPMS:LI30:201:TMITCUS1H") / 1e9

    losses = epics.caget_many(hard_loss_PVS)
    #data["TOTAL_SOFT_LOSSES"] = np.sum(losses[:len(soft_loss_PVS)])
    #data["TOTAL_HARD_LOSSES"] = np.sum(losses)


    data["GDET"] = np.nanmean(obj0)
    data["SIOC:SYS0:ML07:AO737"] = obj2
    data["SIOC:SYS0:ML07:AO736"] = obj1
    data["obj_sum"]= obj_sum
    data["GDET_constraint_check"] = np.nanmean(obj0)
    data["obj_sum_constraint"]= obj_sum
    data["SIOC:SYS0:ML07:AO737_constraint"] = obj2
    data["SIOC:SYS0:ML07:AO736_constraint"] = obj1

    data["time"] = time.time()

    return data

## Create Xopt object

In [ ]:
from xopt import VOCS
# define YAML

Y ="""
variables:
    TDLY:LI21:1:ADelaySet: [0,1] #will update later
    MKB:SYS0:3:VAL: [0,1]
    TDLY:LI21:1:CDelaySet: [0,1]
    MKB:SYS0:4:VAL: [0,1]
    #will add more automatically below


objectives:
    obj_sum: MAXIMIZE

constraints:
    GDET_constraint_check: [GREATER_THAN, 0.001]
    SIOC:SYS0:ML07:AO737_constraint: [GREATER_THAN, 0.001]
    SIOC:SYS0:ML07:AO736_constraint: [GREATER_THAN, 0.001]
"""

vocs = VOCS.from_yaml(Y)

In [ ]:
from xopt import Evaluator, Xopt
from xopt.generators import ExpectedImprovementGenerator

evaluator = Evaluator(function=do_measurement)
generator = ExpectedImprovementGenerator(
    vocs=vocs, turbo_controller="optimize"
)
# for experiments
generator.model_constructor.use_low_noise_prior = False

X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)
X


In [ ]:
## generate random initial samples near current set point
from xopt.utils import get_local_region

init_region = get_local_region(current_values, vocs, fraction=0.1)

# evaluate random points near current setting
X.random_evaluate(3, custom_bounds=init_region)


## run optimization

In [ ]:
n_steps = 10
for i in range(n_steps):
    X.step()

## plot results

In [ ]:
# plot variables
X.vocs.normalize_inputs(X.data).plot(X.vocs.variable_names)

In [ ]:
# plot objective
X.data.plot(X.vocs.objective_names)